# SMMALA sampling of MLP weights using iris data

Sampling the weights of a multi-layer perceptron (MLP) using the iris data and SMMALA.

In [ ]:
## Import packages

import numpy as np

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.distributions import Normal

from eeyore.api import indexify
from eeyore.data import Iris
from eeyore.stats import softabs
from eeyore.models import mlp
from eeyore.mcmc import SMMALA

from timeit import default_timer as timer
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## Avoid issuing memory warning due to number of plots

plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
## Set device to be GPU

device = 'cuda:0'

In [ ]:
## Load iris data

iris = indexify(Iris)()
dataloader = DataLoader(iris, batch_size=150, shuffle=True)

In [ ]:
## Setup MLP model

hparams = mlp.Hyperparameters(dims=[4, 3, 3], activations=[torch.sigmoid, None])
model = mlp.MLP(
    hparams=hparams,
    loss=lambda x, y: nn.CrossEntropyLoss(reduction='sum')(x, torch.argmax(y, 1)),
    dtype=torch.float64,
    device=device
)
model.prior = Normal(
    torch.zeros(model.num_params(), dtype=model.dtype, device=device),
    np.sqrt(3)*torch.ones(model.num_params(), dtype=model.dtype, device=device)
)

In [ ]:
## Setup SMMALA sampler

theta0 = model.prior.sample()
sampler = SMMALA(model, theta0, dataloader, step=0.025, transform=lambda hessian: softabs(hessian, a=10000.))

In [ ]:
## Run SMMALA sampler

start_time = timer()

sampler.run(num_iterations=1100, num_burnin=100)

end_time = timer()
print("Time taken: {}".format(timedelta(seconds=end_time-start_time)))

In [ ]:
## Compute acceptance rate

sampler.chain.acceptance_rate()

In [ ]:
## Compute Monte Carlo mean

sampler.chain.mean()

In [ ]:
## Plot traces of simulated Markov chain

for i in range(model.num_params()):
    chain = sampler.chain.get_theta(i)
    plt.figure()
    sns.lineplot(range(len(chain)), chain)
    plt.xlabel('Iteration')
    plt.ylabel('Parameter value')
    plt.title(r'Traceplot of parameter {}'.format(i+1))

In [ ]:
## Plot running means of simulated Markov chain

for i in range(model.num_params()):
    chain = sampler.chain.get_theta(i)
    chain_mean = torch.empty(len(chain))
    chain_mean[0] = chain[0]
    for j in range(1, len(chain)):
        chain_mean[j] = (chain[j]+j*chain_mean[j-1])/(j+1)
        
    plt.figure()
    sns.lineplot(range(len(chain)), chain_mean)
    plt.xlabel('Iteration')
    plt.ylabel('Parameter value')
    plt.title(r'Running mean of parameter {}'.format(i+1))

In [ ]:
## Plot histograms of simulated Markov chain

for i in range(model.num_params()):
    plt.figure()
    sns.distplot(sampler.chain.get_theta(i), bins=20, norm_hist=True)
    plt.xlabel('Value range')
    plt.ylabel('Relative frequency')
    plt.title(r'Histogram of parameter {}'.format(i+1))